<a href="https://colab.research.google.com/github/CidClayQuirino/ConditionMonitoring_RNN/blob/main/DeslocamentoBrazauro02Set2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install markdown
!pip install statsmodels
!pip install scikit-learn
!pip install PyGithub
!pip install gitpython
!pip install statsmodels
!pip install dash
!pip install xlwt
!pip install openpyxl
!pip install tensorflow
!pip install scipy

In [2]:
import pandas as pd
import requests
from io import BytesIO

# URL do arquivo no GitHub
url = "https://github.com/CidClayQuirino/ConditionMonitoring_RNN/raw/main/faultCodesHistoryBrazauro.xlsx"

# Fazer a requisição HTTP para obter o arquivo
response = requests.get(url)
if response.status_code == 200:
    print("Arquivo baixado com sucesso.")
    # Ler o arquivo Excel em um DataFrame
    df = pd.read_excel(BytesIO(response.content), engine='openpyxl')
    print("Arquivo Excel carregado com sucesso.")
    # Manter apenas as colunas necessárias e renomeá-las
    colunas_selecionadas = ['Asset Serial Number', 'Make', 'Model', 'Name', 'Fault Code', 'Fault Code Cid', 'Fault Code Fmi', 'Fault Code Eid', 'Severity','Fault Date', 'Occurrences','Lifetime Fuel Consumed Value', 'Hours Value', 'Odometer Value', 'Type']
    dfFaultCodesHistoryBrazauro = df[colunas_selecionadas].rename(columns={
        'Asset Serial Number': 'SNum',
        'Make': 'Fabricante',
        'Name': 'NumFrota',
        'Fault Code': 'FaultCode',
        'Fault Code Cid': 'Cid',
        'Fault Code Fmi': 'Fmi',
        'Fault Code Eid': 'Eid',
        'Severity': 'Severity',
        'Fault Date': 'FaultDate',
        'Model': 'Modelo',
        'Occurrences': 'Occurrences',
        'Lifetime Fuel Consumed Value': 'FuelBurn',
        'Hours Value': 'HoursValue',
        'Type': 'Tipo'
    })
dfFaultCodesHistoryBrazauro['FaultDate'] = pd.to_datetime(dfFaultCodesHistoryBrazauro['FaultDate'])
    # Substituir valores na coluna 'Modelo'
dfFaultCodesHistoryBrazauro['Modelo'] = dfFaultCodesHistoryBrazauro['Modelo'].replace('777-05', '777E')

# Extrair apenas a parte da data
dfFaultCodesHistoryBrazauro['FaultDate'] = dfFaultCodesHistoryBrazauro['FaultDate'].dt.date
dfFaultCodesHistoryBrazauro.head()

Arquivo baixado com sucesso.


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Arquivo Excel carregado com sucesso.


,SNum,Fabricante,Modelo,NumFrota,FaultCode,Cid,Fmi,Eid,Severity,FaultDate,Occurrences,FuelBurn,HoursValue,Odometer Value,Tipo
0,KYD02702,CAT,777E,040-002,Low Auto Lube Pressure,NaN,NaN,334.0,2,2024-08-26,127,132586.161233,8761.616454,152794.880352,Event
1,3F300703,CAT,730,060-002,Operator Seat Belt Switch: Voltage Above Normal,3797.0,3.0,NaN,1,2024-08-26,255,45574.469477,8754.130059,37338.307664,Diagnostic
2,3F300703,CAT,730,060-002,Operator Seat Belt Unfastened While Machine is...,NaN,NaN,1388.0,1,2024-08-26,122,45574.469477,8754.066448,37337.686293,Event
3,3F300703,CAT,730,060-002,Real Time Clock Incorrect,NaN,NaN,1320.0,1,2024-08-26,126,45574.469477,8753.770336,37335.449358,Event
4,7W802868,CAT,777E,040-012,"Operator Seat Belt Switch: Erratic, Intermitte...",3797.0,2.0,NaN,1,2024-08-26,127,86683.566964,6992.981983,36651.630572,Diagnostic


In [3]:
# Lista dos modelos a serem removidos
models_to_remove = ["313D", "966", "980", "C15", "CS54B"]

# Filtrar o DataFrame para manter apenas as linhas em que o modelo não está na lista de modelos a serem removidos
dfFaultCodesHistoryBrazauro = dfFaultCodesHistoryBrazauro[~dfFaultCodesHistoryBrazauro['Modelo'].isin(models_to_remove)]

# Exibir o DataFrame atualizado
dfFaultCodesHistoryBrazauro.head()

,SNum,Fabricante,Modelo,NumFrota,FaultCode,Cid,Fmi,Eid,Severity,FaultDate,Occurrences,FuelBurn,HoursValue,Odometer Value,Tipo
0,KYD02702,CAT,777E,040-002,Low Auto Lube Pressure,NaN,NaN,334.0,2,2024-08-26,127,132586.161233,8761.616454,152794.880352,Event
1,3F300703,CAT,730,060-002,Operator Seat Belt Switch: Voltage Above Normal,3797.0,3.0,NaN,1,2024-08-26,255,45574.469477,8754.130059,37338.307664,Diagnostic
2,3F300703,CAT,730,060-002,Operator Seat Belt Unfastened While Machine is...,NaN,NaN,1388.0,1,2024-08-26,122,45574.469477,8754.066448,37337.686293,Event
3,3F300703,CAT,730,060-002,Real Time Clock Incorrect,NaN,NaN,1320.0,1,2024-08-26,126,45574.469477,8753.770336,37335.449358,Event
4,7W802868,CAT,777E,040-012,"Operator Seat Belt Switch: Erratic, Intermitte...",3797.0,2.0,NaN,1,2024-08-26,127,86683.566964,6992.981983,36651.630572,Diagnostic


In [4]:
import plotly.express as px
import pandas as pd

# Agrupar os dados por modelo e data da falha e calcular a soma das ocorrências
grouped = dfFaultCodesHistoryBrazauro.groupby(['Modelo', 'FaultDate']).agg({'Occurrences': 'sum'}).reset_index()

# Criar um gráfico de dispersão interativo para cada modelo
for model, data in grouped.groupby('Modelo'):
    fig = px.scatter(data, x='FaultDate', y='Occurrences', title=f'Disposição das ocorrências ao longo do tempo para o modelo {model}',
                     labels={'FaultDate': 'Data da Falha', 'Occurrences': 'Total de Ocorrências'},
                     hover_data=['Occurrences'])
    fig.show()

In [5]:
import plotly.express as px

# Filtrar o DataFrame para manter apenas as colunas desejadas e remover linhas com valores ausentes em 'Fault Date' e 'Occurrences'
df_filtered = dfFaultCodesHistoryBrazauro[['FaultDate', 'Modelo', 'Occurrences', 'Severity']].dropna(subset=['FaultDate', 'Occurrences'])

# Criar o gráfico de dispersão interativo
fig = px.scatter(df_filtered, x='FaultDate', y='Occurrences', color='Severity',
                 facet_col='Modelo', facet_col_wrap=3,
                 labels={'FaultDate': 'Data da Falha', 'Occurrences': 'Ocorrências', 'Severity': 'Severidade'},
                 title='Dispersão de Ocorrências de Falha por Tempo e Modelo com Indicação de Severidade',
                 color_continuous_scale=['green', 'yellow', 'red'],
                 render_mode='auto')

# Atualizar o layout para ajustar o tamanho dos subplots e adicionar títulos aos subplots
fig.update_layout(height=800, width=1000, showlegend=True, coloraxis_colorbar=dict(title='Severidade'))

# Exibir o gráfico
fig.show()


In [6]:
import plotly.express as px

# Filtrar o DataFrame para manter apenas as ocorrências com Severity igual a 3
df_filtered = dfFaultCodesHistoryBrazauro[dfFaultCodesHistoryBrazauro['Severity'] == 3]

# Drop linhas com valores ausentes em 'Fault Date' e 'Occurrences'
df_filtered = df_filtered.dropna(subset=['FaultDate', 'Occurrences'])

# Calcular a quantidade total de eventos
total_events = df_filtered['Occurrences'].sum()

# Criar o gráfico de dispersão interativo
fig = px.scatter(df_filtered, x='FaultDate', y='Occurrences', color_discrete_sequence=['red'],
                 facet_col='Modelo', facet_col_wrap=3,
                 labels={'FaultDate': 'Data da Falha', 'Occurrences': 'Ocorrências', 'Severity': 'Severidade'},
                 title=f'Dispersão de Ocorrências de Falha por Tempo e Modelo com Severidade 3\nTotal de Eventos: {total_events}')

# Atualizar o layout para ajustar o tamanho dos subplots e adicionar títulos aos subplots
fig.update_layout(height=800, width=1000, showlegend=False)

# Exibir o gráfico
fig.show()


In [7]:
import pandas as pd

# Filtrar o DataFrame para manter apenas as colunas necessárias
df_filtered = dfFaultCodesHistoryBrazauro[['Modelo', 'Severity', 'Occurrences']]

# Criar a tabela pivô
pivot_table = pd.pivot_table(df_filtered, values='Occurrences', index='Modelo', columns='Severity', aggfunc='sum', fill_value=0)

# Adicionar uma coluna com o total de ocorrências por modelo
pivot_table['Total'] = pivot_table.sum(axis=1)

# Adicionar colunas com a porcentagem de ocorrências por severidade
for severity in pivot_table.columns[:-1]:  # Excluir a última coluna 'Total'
    pivot_table[f'%_{severity}'] = (pivot_table[severity] / pivot_table['Total']) * 100

# Arredondar os valores das colunas de porcentagem
pivot_table['%_1'] = pivot_table['%_1'].round(1)
pivot_table['%_2'] = pivot_table['%_2'].round(1)
pivot_table['%_3'] = pivot_table['%_3'].round(1)

# Exibir a tabela
#print(pivot_table)
# Converter a tabela pivô em um DataFrame
dfResumo = pd.DataFrame(pivot_table)

# Exibir o DataFrame
print(dfResumo)

Severity         1        2       3    Total   %_1   %_2   %_3
Modelo                                                        
150-14AWD     6948      615     141     7704  90.2   8.0   1.8
16          265546   138519   11306   415371  63.9  33.3   2.7
352          16831      399       1    17231  97.7   2.3   0.0
395         366800   561672    1865   930337  39.4  60.4   0.2
6030         25390   349519  196166   571075   4.4  61.2  34.4
730        2784417    41792    2478  2828687  98.4   1.5   0.1
777E       5323957  1520344   65029  6909330  77.1  22.0   0.9
D6T          18073    74504       6    92583  19.5  80.5   0.0
D9          975205   151623      42  1126870  86.5  13.5   0.0


In [8]:
import plotly.graph_objects as go

# Supondo que dfResumo seja o DataFrame com as colunas %_1, %_2 e %_3

# Criando os dados para o gráfico
modelos = dfResumo.index.tolist()  # Supondo que os modelos estejam no índice do DataFrame
percentuais_1 = dfResumo['%_1']
percentuais_2 = dfResumo['%_2']
percentuais_3 = dfResumo['%_3']

# Criando as barras com cores personalizadas
fig = go.Figure()
fig.add_trace(go.Bar(x=modelos, y=percentuais_1, text=percentuais_1, textposition='auto', name='%_1', marker_color='green'))
fig.add_trace(go.Bar(x=modelos, y=percentuais_2, text=percentuais_2, textposition='auto', name='%_2', marker_color='yellow'))
fig.add_trace(go.Bar(x=modelos, y=percentuais_3, text=percentuais_3, textposition='auto', name='%_3', marker_color='red'))

# Atualizando o layout do gráfico
fig.update_layout(
    title='Percentuais por Modelo',
    xaxis=dict(title='Modelo'),
    yaxis=dict(title='Percentual'),
    barmode='group'
)

# Exibindo o gráfico interativo
fig.show()


In [9]:
import pandas as pd
#dfFaultCode6030 = dfFaultCode[dfFaultCode['Model'] == "6030"]
dfFaultCode6030 = dfFaultCodesHistoryBrazauro[dfFaultCodesHistoryBrazauro['Modelo'] == "6030"]
dfFaultCode6030 = dfFaultCode6030[dfFaultCode6030['Severity'] > 2]
dfFaultCode6030.head()

,SNum,Fabricante,Modelo,NumFrota,FaultCode,Cid,Fmi,Eid,Severity,FaultDate,Occurrences,FuelBurn,HoursValue,Odometer Value,Tipo
239,6P300111,CAT,6030,020-002,Fuse - VIM / IP ON Drive Control LH CPU (E.C. ...,NaN,NaN,2043.0,3,2024-08-25,1,NaN,4975.003980,NaN,TPIEvents
798,6P300111,CAT,6030,020-002,Engine RH FaultCode 1 (E.C. 5305),NaN,NaN,8376.0,3,2024-08-23,1,NaN,4931.213945,NaN,TPIEvents
830,6P300111,CAT,6030,020-002,LH Shut Off Valve Hydraulic Oil Tank Left Clos...,NaN,NaN,2151.0,3,2024-08-23,2,NaN,4931.068389,NaN,TPIEvents
831,6P300111,CAT,6030,020-002,LH - Fuse VIQ drive control I/O (E.C. 2970),NaN,NaN,2154.0,3,2024-08-23,1,NaN,4931.068389,NaN,TPIEvents
832,6P300111,CAT,6030,020-002,Cable break at servo control pressure (E.C. 1780),NaN,NaN,2015.0,3,2024-08-23,2,NaN,4931.068389,NaN,TPIEvents


In [10]:
import pandas as pd

# Suponha que dfFaultCode6030 seja o DataFrame original com as colunas 'Fault Code' e 'Occurrences'

# Agrupar por 'Fault Code' e somar as ocorrências
df_fault_code_summary = dfFaultCodesHistoryBrazauro.groupby('FaultCode')['Occurrences'].sum().reset_index()

# Ordenar o DataFrame em ordem decrescente de ocorrências, se necessário
df_fault_code_summary_sorted = df_fault_code_summary.sort_values(by='Occurrences', ascending=False)

# Exibir o DataFrame resultante
print(df_fault_code_summary_sorted)


                                             FaultCode  Occurrences
408    Operator Seat Belt Switch: Voltage Above Normal      2175272
407  Operator Seat Belt Switch: Erratic, Intermitte...      1485280
417                      Positive High Peak Frame Bias      1342363
402                      Negative High Peak Frame Rack       981280
400                      Negative High Peak Frame Bias       965433
..                                                 ...          ...
406      Operator In Seat Sensor: Voltage Below Normal            1
278  Hydraulic Oil Cooling Fan Solenoid: Voltage Ab...            1
169  Defective pressure reducing valve 50bar or def...            1
389  Main Hydraulic Pump #1 Solenoid: Voltage Above...            1
0    10 Volt Sensor DC Power Supply: Voltage Above ...            1

[599 rows x 2 columns]


In [11]:

import plotly.express as px

# Supondo que você tenha um DataFrame chamado dfFaultCode6030 com os dados

# Criar o gráfico de dispersão
fig = px.scatter(dfFaultCode6030, x='FaultDate', y='Occurrences', color='Occurrences',
                 color_continuous_scale='Reds', title='Gráfico de Dispersão - Occurrences por Fault Date & Severity = 3',
                 facet_col='SNum', facet_col_wrap=3)  # facet_col_wrap define quantos gráficos são exibidos por linha

# Exibir o gráfico
fig.show()


In [12]:
import pandas as pd
#dfFaultCode6030 = dfFaultCode[dfFaultCode['Model'] == "6030"]
dfFaultCode777E = dfFaultCodesHistoryBrazauro[dfFaultCodesHistoryBrazauro['Modelo'] == "777E-05"]
#dfFaultCode6030 = dfFaultCode6030[dfFaultCode6030['Severity'] >= 1]
dfFaultCode777E.head()

,SNum,Fabricante,Modelo,NumFrota,FaultCode,Cid,Fmi,Eid,Severity,FaultDate,Occurrences,FuelBurn,HoursValue,Odometer Value,Tipo


In [13]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Supondo que dfFaultCode777E seja seu DataFrame original
# dfFaultCode777E = pd.read_csv('seu_arquivo.csv')  # Exemplo de carregamento

# Criação de uma lista com Serial Numbers únicos para a dropdown
serial_numbers = dfFaultCodesHistoryBrazauro['SNum'].unique()

# Criação do aplicativo Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    # Dropdown para selecionar o Serial Number
    dcc.Dropdown(
        id='serial-dropdown',
        options=[{'label': sn, 'value': sn} for sn in serial_numbers],
        value=serial_numbers[0]  # Valor inicial do dropdown
    ),
    # Gráfico para mostrar os dados
    dcc.Graph(id='fault-code-graph')
])

@app.callback(
    Output('fault-code-graph', 'figure'),
    [Input('serial-dropdown', 'value')]
)
def update_graph(selected_serial):
    df_filtered = dfFaultCodesHistoryBrazauro[dfFaultCodesHistoryBrazauro['SNum'] == selected_serial]

    # Calcular a quantidade total de ocorrências para cada Fault Code
    fault_code_occurrences = df_filtered.groupby('FaultCode')['Occurrences'].sum()

    # Selecionar os top 10 Fault Codes com base na quantidade de ocorrências
    top_10_fault_codes = fault_code_occurrences.nlargest(10).index.tolist()

    # Filtrar o DataFrame original para incluir apenas os dados relacionados aos top 10 Fault Codes
    df_top_10_fault_codes = df_filtered[df_filtered['FaultCode'].isin(top_10_fault_codes)]

    # Criar o gráfico de dispersão
    fig = px.scatter(df_top_10_fault_codes, x='FaultDate', y='Occurrences', color='FaultCode',
                     title=f'Gráfico de Dispersão dos Top 10 Fault Codes por Data de Falha {selected_serial}',
                     labels={'Occurrences': 'Quantidade de Ocorrências', 'FaultDate': 'Data de Falha'},
                     hover_data={'FaultCode': True, 'Occurrences': True})

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<ipython-input-13-48227c3f1e23>:4: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

<ipython-input-13-48227c3f1e23>:5: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



<IPython.core.display.Javascript object>

In [15]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Supondo que dfFaultCode777E seja seu DataFrame original
# dfFaultCode777E = pd.read_csv('seu_arquivo.csv')  # Exemplo de carregamento

# Criação de uma lista combinando Serial Number e NumFrota para a dropdown
dfFaultCodesHistoryBrazauro['SerialNumFrota'] = dfFaultCodesHistoryBrazauro['SNum'] + ' - ' + dfFaultCodesHistoryBrazauro['NumFrota'].astype(str)
serial_num_frota = dfFaultCodesHistoryBrazauro['SerialNumFrota'].unique()

# Criação do aplicativo Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    # Dropdown para selecionar o Serial Number e NumFrota
    dcc.Dropdown(
        id='serial-dropdown',
        options=[{'label': snf, 'value': snf} for snf in serial_num_frota],
        value=serial_num_frota[0]  # Valor inicial do dropdown
    ),
    # Gráfico para mostrar os dados
    dcc.Graph(id='fault-code-graph')
])

@app.callback(
    Output('fault-code-graph', 'figure'),
    [Input('serial-dropdown', 'value')]
)
def update_graph(selected_serial_frota):
    # Dividir o valor selecionado para obter SNum e NumFrota
    selected_serial, selected_frota = selected_serial_frota.split(' - ')

    df_filtered = dfFaultCodesHistoryBrazauro[
        (dfFaultCodesHistoryBrazauro['SNum'] == selected_serial) &
        (dfFaultCodesHistoryBrazauro['NumFrota'].astype(str) == selected_frota)
    ]

    # Calcular a quantidade total de ocorrências para cada Fault Code
    fault_code_occurrences = df_filtered.groupby('FaultCode')['Occurrences'].sum()

    # Selecionar os top 10 Fault Codes com base na quantidade de ocorrências
    top_10_fault_codes = fault_code_occurrences.nlargest(10).index.tolist()

    # Filtrar o DataFrame original para incluir apenas os dados relacionados aos top 10 Fault Codes
    df_top_10_fault_codes = df_filtered[df_filtered['FaultCode'].isin(top_10_fault_codes)]

    # Criar o gráfico de dispersão
    fig = px.scatter(df_top_10_fault_codes, x='FaultDate', y='Occurrences', color='FaultCode',
                     title=f'Gráfico de Dispersão dos Top 10 Fault Codes por Data de Falha {selected_serial_frota}',
                     labels={'Occurrences': 'Quantidade de Ocorrências', 'FaultDate': 'Data de Falha'},
                     hover_data={'FaultCode': True, 'Occurrences': True})

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>